In [29]:
import pandas as pd
import numpy as np
import time
import re

In [31]:
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from datetime import datetime
from selenium.webdriver.common.keys import Keys

from bs4 import BeautifulSoup as soup

global driver

options = Options()

#  Code to disable notifications pop up of Chrome Browser
options.add_argument("--disable-notifications")
options.add_argument("--disable-infobars")
options.add_argument("--mute-audio")
# options.add_argument("headless")

cr_option = webdriver.ChromeOptions()
chrome_prefs = {}
cr_option.experimental_options["prefs"] = chrome_prefs
chrome_prefs["profile.default_content_settings"] = {"images": 2}
chrome_prefs["profile.managed_default_content_settings"] = {"images": 2}

driver = webdriver.Chrome(executable_path=r'chromedriver.exe', options=options, chrome_options=cr_option)

C:\Users\alim.hanif\AppData\Local\Continuum\anaconda3\envs\acube\lib\site-packages\ipykernel_launcher.py:28: DeprecationWarning: use options instead of chrome_options


In [32]:
def scrape_gsm(raw_keyword_):
    raw_keyword = raw_keyword_
    driver.get("https://google.com")

    element = driver.find_element_by_name("q")
    element.clear()
    element.send_keys(raw_keyword)

    driver.find_element_by_name("q").send_keys(Keys.RETURN)

    try:
        c=driver.find_elements_by_xpath("//a[@class='gL9Hy']")
        rec_keyword = soup(c[0].get_attribute('innerHTML')).text
    except:
        driver.get("https://en.wikipedia.org/wiki/Main_Page")
        driver.get("https://google.com")
        print(raw_keyword)
        input('pause')
        c=driver.find_elements_by_xpath("//a[@class='gL9Hy']")
        rec_keyword = soup(c[0].get_attribute('innerHTML')).text

    c = driver.find_elements_by_xpath("//div[@class='r']")
    for i in c:
        page_soup = soup(i.get_attribute('innerHTML'))
        href = page_soup.a['href']
        if 'gsmarena' in href and 'compare' not in href:
            # print(href)
            break
    return raw_keyword_, rec_keyword, href
    


In [38]:
df_cek = pd.DataFrame({'type': ['xiaomi note 5']})

In [39]:
data = []
data_href = []
count = 1
df_r = pd.DataFrame()

for index, row in df_cek.iterrows():
    df_temp = pd.DataFrame()
    tipe_hp = '-'
    type_ = row['type']
    if type_ != '-':
        raw_keyword, rec_keyword, href = scrape_gsm(type_+' gsmarena')
        if href not in data_href:
            data_href.append(href)
            driver.get(href)
            time.sleep(1)
            try:
                c=driver.find_element_by_xpath("//h1[@class='specs-phone-name-title']")
                tipe_hp = re.sub(r'\s+', ' ', soup(c.get_attribute('innerHTML')).text)
                df_page = pd.read_html(driver.page_source)
                for i in range(len(df_page)):
                    df_temp = df_temp.append(df_page[i])
                df_temp = df_temp.dropna()
                df_temp['type']= tipe_hp
                df_r = df_r.append(df_temp)
            except:
                tipe_hp = '--'
        else:
            tipe_hp = '-'
    else:
        raw_keyword, rec_keyword, href = '','',''
    data.append({'raw_keyword':type_, 'rec_keyword': rec_keyword,'type': tipe_hp, 'href':href})
    print(str(count)+'/'+str(len(df_cek))+' '+raw_keyword)
    count += 1

1/1 xiaomi note 5 gsmarena


In [40]:
df_result = pd.DataFrame(data)
# df_result.to_csv('df_hp_with_specs.csv')
df_result

,href,raw_keyword,rec_keyword,type
0,https://www.gsmarena.com/xiaomi_redmi_note_5_a...,xiaomi note 5,,Xiaomi Redmi Note 5 AI Dual Camera
